<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/%E9%89%85%E4%BA%A8%E7%B6%B2%E6%96%87%E5%AD%97%E6%8E%A2%E5%8B%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture
!pip install transformers
!pip install gdown
!pip install opencc-python-reimplemented
!pip install googletrans==3.1.0a0
!pip install fake_useragent

In [119]:
import torch
import numpy as np
import pandas as pd

In [7]:
from transformers import BertModel, BertTokenizer
PATH = '/content/FinBERT_L-12_H-768_A-12_pytorch'

In [15]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent


In [144]:
import time
import datetime as dt

import time
import tqdm

In [6]:
from opencc import OpenCC
cc = OpenCC('t2s')

In [ ]:
#!gdown https://drive.google.com/uc?id=1qW1YWtw3q9Q28QThrIY-rDU9Gl-SLIKO
#!unzip /content/FinBERT_L-12_H-768_A-12_pytorch.zip

# 財金BERT

In [ ]:


tokenizer = BertTokenizer.from_pretrained(PATH ,local_files_only=True)
model = BertModel.from_pretrained(PATH ,local_files_only=True)

In [ ]:
sector_dict = {
'健護': '醫療保健服務類股',
'科技': '科技類股', 
'公用事業': '公用事業類股',
'生技':'生技類股', 
'能源':'能源類股',
'替代能源':'替代能源類股',
'消費':'消費品類股',
'金融':'金融類股', 
'礦業':'礦業',
'天然資源':'天然資源', 
'貴金屬現貨':'貴金屬',
'農產品':'農產品',
'貴金屬類股':'貴金屬',
}

In [ ]:
mkt_dict= [
'新興市場債',
'高收益債',
'全球債',
'投資等級公司債',
'美國公債',
'亞債',
'REITs不動產投資信託',
'新臺幣類貨幣',
'外幣類貨幣',
'亞太股市', '美國股市', '全球新興市場', 
'全球型股市', '大中華股市', '中國A股', '印度股市','歐洲股市', 
'東南亞股市', '拉丁美洲股市', 
'韓國股市', '新興歐洲股市', '德國股市', '台灣股市','俄羅斯股市',
'中國H股', '日本股市', '英國股市','泰國股市',
'香港股市', '澳洲股市', '印尼股市', '巴西股市', '歐非中東股市'
]


In [ ]:
mkt_dict = {x:x for x in mkt_dict}
mkt_dict['亞債'] = '亞洲債券'
mkt_dict['REITs不動產投資信託'] = '不動產'

In [ ]:
total_sec_dict = {**sector_dict, **mkt_dict}

In [ ]:
sentences = list(set(total_sec_dict.values()))
sentences = [cc.convert(x) for x in sentences]

In [ ]:
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [ ]:
## 單純forward pass 沒要計算梯度
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
x = outputs['last_hidden_state']
np.savetxt('bert_mkt_emb.tsv', x[:,0].cpu().detach().numpy(), delimiter="\t")
pd.DataFrame(sentences).to_csv('bert_mkt_emb_meta.tsv', header=False, index=False, sep="\t")

# 鉅亨網爬蟲


In [16]:
res = requests.get('https://news.cnyes.com/news/cat/wd_stock')

In [10]:
#res2 = requests.get('https://blog.anuefund.com/page/2/?utm_campaign=bloglink&utm_medium=fund_mainlist&utm_source=anuefundapp')

In [11]:
soup = BeautifulSoup(res.text, 'html.parser')

In [12]:
news_list = soup.find_all("a", attrs={"class":"_1Zdp"})

In [ ]:
[x['href'] for x in news_list]
[x.find('h3').get_text(strip=True) for x in news_list]

## .get_text(strip=True) 可避免不必用空白符亂碼

In [17]:
ua = UserAgent()
headers = {'user-agent': ua.random}

In [145]:
start_unix_timestamp = int(time.mktime(dt.date(2021,1,1).timetuple()))
end_unix_timestamp = int(time.mktime(dt.date(2021,11,15).timetuple()))

In [149]:
url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(start_unix_timestamp, end_unix_timestamp, 1)
res = requests.get(url_api, headers=headers)
res_json = res.json()

In [147]:
last_page = res_json['items']['last_page']

In [148]:
last_page

613

In [ ]:
news_data_list

In [ ]:
news_data_list = []

for i in tqdm.tqdm(range(1,last_page+1)):
  url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(start_unix_timestamp, end_unix_timestamp, i)
  res = requests.get(url_api, headers=headers)
  res_json = res.json()
  news_data_list.append(res_json)
  time.sleep(1)

  2%|▏         | 14/613 [00:27<18:13,  1.83s/it]

In [65]:
import re

In [112]:
## 只保留英文中文數字 

def remove_punctuation(line):
  rule = re.compile("[^a-z A-Z \u4e00-\u9fa5]")
  #rule = re.compile("[^a-z A-Z 0-9 \u4e00-\u9fa5]")
  line = rule.sub('',line)
  return line

In [113]:
remove_punctuation('')

''

In [52]:
from datetime import datetime
ts = int('1284101485')

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(dt.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2010-09-10 06:51:25


In [ ]:
news_data_list[0]

In [131]:

news_extracted_list=[]

for chunk in news_data_list:
  article_content = chunk['items']['data']
  data_dict = [{
      'id':x['newsId'], 
      'title':x['title'], 
      'summary':x['summary'],  
      'unix_timestamp':x['publishAt'],
      'datetime':dt.datetime.utcfromtimestamp(x['publishAt']).strftime('%Y-%m-%d')
      } for x in article_content
  ]
  news_extracted_list.extend(data_dict)


In [ ]:
news_data_list[19]

In [133]:
pd.DataFrame(news_extracted_list)

,id,title,summary,unix_timestamp,datetime
0,4771793,國產出售高雄悅誠廣場資產 貢獻EPS 1.27元,混凝土廠國產 (2504-TW) 今 (15) 日公告，將高雄「悅誠廣場」房地持分以總價 2...,1636984978,2021-11-15
1,4771790,〈台泥法說〉Q4煤價下滑成本壓力減 毛利率將回升至平均之上,台泥 (1101-TW) 今 (15) 日召開法說，總經理李鐘培表示，10 月以來中國能耗雙...,1636984833,2021-11-15
2,4771785,〈台泥法說〉張安平談氣候變遷「水泥業站在生命第一現場」 承擔減碳責無旁貸,台泥 (1101-TW) 董事長張安平今 (15) 日在法說會前致詞時指出，今天的氣候災害，...,1636984795,2021-11-15
3,4771771,達邁調整產品組合 看好高頻高速傳輸及汽車市場應用,聚醯亞胺（PI）薄膜廠達邁 (3645-TW) 積極展開產品應用多元化的布局，由以往高達 9...,1636980649,2021-11-15
4,4771770,22家壽險前3季合賺3389億元破紀錄 淨值比all pass！,22 家壽險前 3 季合賺 3389 億元破紀錄 淨值比 all pass！,1636980334,2021-11-15
...,...,...,...,...,...
585,4762178,權證投資人一定要了解的權證投資人看多標的TOP5,,1636076093,2021-11-05
586,4762084,Dish Network用戶數下滑+5G佈建遙遙無期 股價重挫近14%,衛星電視供應商 Dish Network (DISH-US) 周四 (4 日) 表示該公司用...,1636075112,2021-11-05
587,4762115,〈台股盤前〉美股續創高待帶量上攻 類股持續輪動,台股昨 (4) 日在權王台積電走弱，貨櫃三雄賣壓湧現下，翻黑下挫，下跌 43.3 點，收 1...,1636072504,2021-11-05
588,4761480,操盤手看台股：輪動格局 缺料緩解股進入第二回合,,1636070423,2021-11-05
